In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np


import torch.nn.functional as F


class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 28*28 // 2)
        self.fc2 = nn.Linear(28*28 // 2, 28*28 // 2 // 2)
        self.fc3 = nn.Linear(28*28 // 2 // 2, 28*28 // 2 // 2 // 2)
        self.fc4 = nn.Linear(28*28 // 2 // 2 // 2, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)

        return x

def calculate_metrics(loader: DataLoader, model: nn.Module):
    y_true = []
    y_pred = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_pred.extend(predicted.numpy())
            y_true.extend(labels.numpy())

    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, average='macro')
    precision = precision_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    return accuracy, recall, precision, f1

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.FashionMNIST(root='./data_dz2', train=True, download=True, transform=transform)
valset = torchvision.datasets.FashionMNIST(root='./data_dz2', train=False, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
valloader = DataLoader(valset, batch_size=64, shuffle=False)


net = SimpleNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)

val_acc, val_rec, val_prec, val_f1 =0,0,0,0
epoch = 0


while val_acc < 0.95 and val_rec < 0.95 and val_prec < 0.95 and val_f1 < 0.95:
    
    running_loss = 0.0
    net.train()
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss:nn.CrossEntropyLoss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(trainloader):.3f}')

    train_acc, train_rec, train_prec, train_f1 = calculate_metrics(trainloader, net)
    print(f'Training - Accuracy: {train_acc}, Recall: {train_rec}, Precision: {train_prec}, F1 Score: {train_f1}')

    val_acc, val_rec, val_prec, val_f1 = calculate_metrics(valloader, net)
    print(f'Validation - Accuracy: {val_acc}, Recall: {val_rec}, Precision: {val_prec}, F1 Score: {val_f1}')

    epoch+=1

print('Finished Training')

Epoch 1, Loss: 1.667
Training - Accuracy: 0.8563, Recall: 0.8546393297736484, Precision: 0.8744462607435839, F1 Score: 0.8487716277655533
Validation - Accuracy: 0.8567, Recall: 0.8556422642609707, Precision: 0.8769210351092591, F1 Score: 0.8491976333162732
Epoch 2, Loss: 1.295
Training - Accuracy: 0.8993666666666666, Recall: 0.8987683230623744, Precision: 0.9017217276831275, F1 Score: 0.8981625616571757
Validation - Accuracy: 0.8999, Recall: 0.8991114574354228, Precision: 0.9014420408973491, F1 Score: 0.8986227531560997
Epoch 3, Loss: 1.125
Training - Accuracy: 0.8719833333333333, Recall: 0.8698886308273218, Precision: 0.8937637021821369, F1 Score: 0.8739529428945836
Validation - Accuracy: 0.8654, Recall: 0.8636788925939205, Precision: 0.8869510898382297, F1 Score: 0.866828520237155
Epoch 4, Loss: 1.217
Training - Accuracy: 0.89455, Recall: 0.8924278266351093, Precision: 0.9022360465433537, F1 Score: 0.8921448778702168
Validation - Accuracy: 0.8898, Recall: 0.888432014232978, Precision